# Analyzing Chicago Census Data, Crime Data and Public Schools Data

This assignment involves 3 datasets for the city of Chicago obtained from the Chicago Data Portal:

**1. Chicago Socioeconomic Indicators**

This [dataset](https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-wwwcourseraorg-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01) contains a selection of six socioeconomic indicators of public health significance and a hardship index, by Chicago community area, for the years 2008 – 2012.

**2. Chicago Public Schools**

This [dataset](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01) shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year.

**3. Chicago Crime Data**

This [dataset](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01) reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.

## Install Libraries

In [71]:
!pip install mysql-connector-python
!pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Import Libraries

In [72]:
import mysql.connector as mysql
from pandasql import sqldf as sql
import pandas as pd

## DB Credentials

In [73]:
db_host = "***"
db_name = 'ibm_database'
db_user = '***'
db_pass = '***'

## Database Connection Function

In [74]:
# Conection
def connect_database(host, username, password, database):
    """
    Connect to database
    Params:
        - host: database host
        - username: database user
        - password: database password
        - database: database name
        
    Returns
        conn: connection with the db
    """
    try:
        # connection function
        conn = mysql.connect(host=host, user=username, password=password, db=database, auth_plugin='mysql_native_password')
        
        print('Connected to database')
        
        return conn
        
    # handles exceptions
    except Exception as e:
        print('Connection failed! Exception: ', e)

# call function connect_database()
conn = connect_database(db_host, db_user, db_pass, db_name)

Connected to database


## Problem 1: Find the total number of crimes recorded in the CRIME table.

In [75]:
%%capture --no-display
pd.read_sql("""
     SELECT COUNT(*) FROM CHICAGO_CRIME_DATA;
""", con=conn)

,COUNT(*)
0,533


## Problem 2: List community areas with per capita income less than 11000.

In [38]:
%%capture --no-display
pd.read_sql("""
    SELECT
        COMMUNITY_AREA_NAME,
        PER_CAPITA_INCOME
    FROM CENSUS_DATA
    WHERE PER_CAPITA_INCOME < 11000;
""", con=conn)

,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
0,West Garfield Park,10934
1,South Lawndale,10402
2,Fuller Park,10432
3,Riverdale,8201


## Problem 3: List all case numbers for crimes involving minors?

In [40]:
%%capture --no-display
pd.read_sql("""
    SELECT
        CASE_NUMBER
    FROM CHICAGO_CRIME_DATA
    WHERE DESCRIPTION LIKE '%MINOR%';
""", con=conn)

,CASE_NUMBER
0,HL266884
1,HK238408


## Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [45]:
%%capture --no-display
pd.read_sql("""
    SELECT
        CASE_NUMBER,
        PRIMARY_TYPE,
        DESCRIPTION
    FROM CHICAGO_CRIME_DATA
    WHERE PRIMARY_TYPE = 'KIDNAPPING';
""", con=conn)

,CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
0,HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


## Problem 5: What kind of crimes were recorded at schools?

In [49]:
%%capture --no-display
pd.read_sql("""
    SELECT
        PRIMARY_TYPE,
        LOCATION_DESCRIPTION
    FROM CHICAGO_CRIME_DATA
    WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'
    GROUP BY PRIMARY_TYPE, LOCATION_DESCRIPTION;
""", con=conn)

,PRIMARY_TYPE,LOCATION_DESCRIPTION
0,BATTERY,"SCHOOL, PUBLIC, GROUNDS"
1,BATTERY,"SCHOOL, PUBLIC, BUILDING"
2,CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
3,NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
4,NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
5,ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
6,CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
7,PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
8,PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


## Problem 6: List the average safety score for all types of schools.

In [50]:
%%capture --no-display
pd.read_sql("""
    SELECT
        AVG(SAFETY_SCORE)
    FROM CHICAGO_PUBLIC_SCHOOLS;
""", con=conn)

,AVG(SAFETY_SCORE)
0,44.8693


## Problem 7: List 5 community areas with highest % of households below poverty line.

In [58]:
%%capture --no-display
pd.read_sql("""
    SELECT
        COMMUNITY_AREA_NAME,
        PERCENT_HOUSEHOLDS_BELOW_POVERTY
    FROM CENSUS_DATA
    ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
    LIMIT 5;
""", con=conn)

,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,Riverdale,56.5
1,Fuller Park,51.2
2,Englewood,46.6
3,North Lawndale,43.1
4,East Garfield Park,42.4


## Problem 8: Which community area(number) is most crime prone?

In [59]:
%%capture --no-display
pd.read_sql("""
    SELECT
        COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) AS Frequency 
    FROM CHICAGO_CRIME_DATA 
    WHERE COMMUNITY_AREA_NUMBER <> 0 
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
    LIMIT 1;
""", con=conn)

,COMMUNITY_AREA_NUMBER,Frequency
0,25,43


## Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [63]:
%%capture --no-display
pd.read_sql("""
    SELECT
        COMMUNITY_AREA_NAME
    FROM CENSUS_DATA
    WHERE HARDSHIP_INDEX = (
        SELECT
            MAX(HARDSHIP_INDEX)
        FROM CENSUS_DATA
    );
""", con=conn)

,COMMUNITY_AREA_NAME
0,Riverdale


## Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [68]:
%%capture --no-display
pd.read_sql("""
    SELECT
        DISTINCT COMMUNITY_AREA_NUMBER,
        COMMUNITY_AREA_NAME
    FROM CENSUS_DATA
    WHERE COMMUNITY_AREA_NUMBER = (
        SELECT
            COMMUNITY_AREA_NUMBER
        FROM CHICAGO_CRIME_DATA
        GROUP BY COMMUNITY_AREA_NUMBER
        ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
        LIMIT 1);
""", con=conn)

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,25,Austin


## Close Database Connection

In [70]:
conn.close()
print('Connection closed!')

Connection closed!
